In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
from datetime import datetime
import time
import ta

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from data.get_data import *

from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [5]:
import ta
from feature.indicator import *
from multiAnalysis.volatility import GARCH

start = "2023-01"
end = "2023-12"

data = Get_data("BTC", "1d")
data = data.loc[start : end]

rets = data.close.pct_change().dropna()*100
rets.name = "returns"

ma_1 = 3
ma_2 = 7
ma_3 = 14

atr_ = 24
rsi_ = 14

window = 10

data['returns'] = data.close.pct_change()

data['ma_1'] = ma(data, ma_1)
data['ma_2'] = ma(data, ma_2)
data['ma_3'] = ma(data, ma_3)

data['vol_std'] = rets.rolling(window).std()
data['vol'] = rets.ewm(span = 5).std()

data['sar'], data['sar_down'], data['sar_up'] = sar(data)

data['garch'] = GARCH(rets)

data['r_target'] = data['returns'].rolling(3).mean()
data['target'] = np.where(data['r_target'] > 0, 1, 0)

                       Zero Mean - GARCH Model Results                        
Dep. Variable:                returns   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.004
Vol Model:                      GARCH   Log-Likelihood:               -613.739
Distribution:                  Normal   AIC:                           1233.48
Method:            Maximum Likelihood   BIC:                           1244.34
                                        No. Observations:                  276
Date:                Fri, Oct 06 2023   Df Residuals:                      276
Time:                        17:18:44   Df Model:                            0
                            Volatility Model                            
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
omega          3.2217      0.864      3.730  1.917e-04 [  1.529,  4.91

In [6]:
from utils.plot import *

fig = make_subplots(rows = 2, cols = 1, shared_xaxes = True, row_heights = [0.8, 0.2], vertical_spacing = 0.01)
#shared_xaxes=True, row_heights=[0.6, 0.2, 0.2]
fig.add_trace(
    go.Candlestick(
        x = data.index , open = data.open, close = data.close,
        high = data.high, low = data.low, name = data.symbol[0],
        yaxis='y1'
    ),
    col = 1, row = 1
)


add_line(fig, x=data.index, y=data['ma_1'], name='ma_1', color='blue')
add_line(fig, x=data.index, y=data['ma_2'], name='ma_2', color='orange')
add_line(fig, x=data.index, y=data['ma_3'], name='ma_3', color='black')
add_mark(fig, x=data.index, y=data['sar_up'], name='sar_up', color='blue')
add_mark(fig, x=data.index, y=data['sar_down'], name='sar_down', color='black')


add_line(fig, x=data.index, y=data['target'], name='target', color='blue', row = 2 , col = 1)
add_line(fig, x=data.index, y=data['vol'], name='vol', color='blue', row = 2 , col = 1)
add_line(fig, x=data.index, y=data['garch'], name='garch', color='green', row = 2 , col = 1)


fig.update_xaxes(rangeslider_visible = False , col =1, row =1)

fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)

#Fig = go.FigureWidget(fig)
fig.update_layout(height = 800 , width =1500,
                  margin=dict(
                        l=5, r=0, b=10, t=30,
                        pad=1
                    ),
                  showlegend=True,
                  )
fig.show()

## Optimiser Triple MM

In [4]:
from sklearn.model_selection import ParameterGrid


In [5]:
MM = {
    'a_low' : [2, 4, 6, 8],
    'b_meduim' : [9, 11, 13, 15],
    'c_high' : [16, 18, 20, 22]
}

#list(ParameterGrid(MM))

# Trend-following

## Channel Breakout

In [6]:
slow_period = 30
fast_period = 14


def calculate_donchian_channels(data, slow_period, fast_period):
    data["slow_high"] = data["high"].rolling(window = slow_period).max()
    data["slow_low"] = data["low"].rolling(window = slow_period).min()
    data["fast_high"] = data["high"].rolling(window = fast_period).max()
    data["fast_low"] = data["low"].rolling(window = fast_period).min()
    
def channel_breakout_strategy(data):
    data["long_entry"] = data["close"] > data["fast_high"]
    data["short_entry"] = data["close"] < data["fast_low"]
    data["long_exit"] = data["close"] < data["slow_low"]
    data["short_exit"] = data["close"] > data["slow_high"]
    

In [7]:
calculate_donchian_channels(data, slow_period, fast_period)

In [8]:
from utils.plot import *

#fig = make_subplots(rows = 2, cols = 1, shared_xaxes = True, row_heights = [0.8, 0.2], vertical_spacing = 0.01)
#shared_xaxes=True, row_heights=[0.6, 0.2, 0.2]
fig.add_trace(
    go.Candlestick(
        x = data.index , open = data.open, close = data.close,
        high = data.high, low = data.low, name = data.symbol[0],
        yaxis='y1'
    ),
    col = 1, row = 1
)


add_line(fig, x=data.index, y=data['slow_high'], name='slow_high', color='blue')
add_line(fig, x=data.index, y=data['slow_low'], name='slow_low', color='orange')
add_line(fig, x=data.index, y=data['fast_high'], name='fast_high', color='black')
add_line(fig, x=data.index, y=data['fast_low'], name='fast_low', color='blue')
#add_mark(fig, x=data.index, y=data['fast_low'], name='fast_low', color='blue')


fig.update_xaxes(rangeslider_visible = False , col =1, row =1)

fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)

#Fig = go.FigureWidget(fig)
fig.update_layout(height = 800 , width =1500,
                  margin=dict(
                        l=5, r=0, b=10, t=30,
                        pad=1
                    ),
                  showlegend=True,
                  )
fig.show()

## Moving Average

In [9]:
def calculate_AMA_AMATR1(prices, alpha, beta):
    ama_values = []
    channels = []
    trend_directions = []
    
    for i in range(len(prices)):
        if i == 0:
            ama = prices[i]
            upper_channel = ama * (1 + beta)
            lower_channel = ama * (1 - beta)
        else:
            ama = alpha * prices[i] + (1 - alpha) * ama
            upper_channel = upper_channel * (1 + beta)
            lower_channel = lower_channel * (1 - beta)
        
        ama_values.append(ama)
        channels.append((upper_channel, lower_channel))
        
        if prices[i] > upper_channel:
            trend_directions.append("Uptrend")
        elif prices[i] < lower_channel:
            trend_directions.append("Downtrend")
        else:
            trend_directions.append("no trend")
        
        return ama_values, channels, trend_directions
    

In [10]:
def calculate_AMA_AMATR2(prices, alpha_min, alpha_max, gamma, beta_min, beta_max):
    ama_values = []
    channels = []
    trend_directions = []
    signal_to_noise_ratios = []
    
    alpha = alpha_min
    beta = beta_min
    
    for i in range(len(prices)):
        if i == 0:
            ama = prices[i]
        else:
            ama = alpha * prices[i] + (1 - alpha) * ama
        
        upper_channel = (1 + beta) * ama
        lower_channel = (1 - beta) * ama
        
        deviation_plus = alpha * max(0, prices[i] - prices[i - 1]) + (1 - alpha) * deviation_plus
        deviation_minus = alpha * max(0, prices[i - 1] - prices[i]) + (1 - alpha) * deviation_minus
        
        snr = (prices[i] - ama) / ama
        
        if prices[i] > upper_channel:
            trend_directions.append("Uptrend")
        elif prices[i] < lower_channel:
            trend_directions.append("Dowtrend")
        else:
            trend_directions.append("No Trend")
        
        ama_values.append(ama)
        channels.append((upper_channel, lower_channel))
        signal_to_noise_ratios.append(snr)
        
        if trend_directions[-1] == "Uptrend":
            alpha = alpha_min + (alpha_max - alpha_min) * np.arctan(gamma * snr)
            beta = beta_max
        elif trend_directions[-1] == "Downtrend":
            alpha = alpha_max + (alpha_max - alpha_min) * np.arctan(gamma * snr)
            beta = beta_max
        else:
            alpha = alpha_max
            beta = beta_min
        
        return ama_values, channels, trend_directions, signal_to_noise_ratios
    

## Swing Breakout

In [11]:
def calculate_swing_breakout_strategy(prices, alpha, beta, T):
    states = []
    positions = []
    profits = []
    
    state = "INITVOL"
    position = 0
    profit = 0
    rolling_volatility = 0
    
    for t in range(len(prices)):
        if state == "INITVOL":
            if t >= T:
                rolling_volatility = np.std(prices[t - T:t])
                S = prices[t] / (1 + rolling_volatility * alpha/2)
                L = prices[t] * (1 + rolling_volatility * alpha/2)
                state = "INIT"
        elif state == "INIT":
            if prices[t] > L:
                state = "LONG"
                position = 1
                PFL = prices[t] * (1 + rolling_volatility * beta)
            elif prices[t] < S:
                state = "SHORT"
                position = -1
                PFS = prices[t] / (1 + rolling_volatility * beta)
        
        elif state == "LONG":
            if prices[t] > PFL:
                state = "INIT"
                position = 0
                S = prices[t] / (1 + rolling_volatility * alpha/2)
                L = prices[t] * (1 + rolling_volatility * alpha/2)
            elif prices[t] > L:
                state = "LONG"
                position = 1
                PFL = prices[t] * (1 + rolling_volatility * beta)
        
        states.append(state)
        positions.append(position)
        profits.append(profit)
        
    return states, positions, profits